In [ ]:
import pandas as pd
import numpy as np
import math
import datetime
import bandit.main as bd

In [ ]:
def get_frame(filename):
    df = pd.read_excel(io=filename)
    # df_combined = pd.merge(df,df_lookup,on='Difficulty')
    # df_combined['Duration'] = df_combined['Time'].apply(lambda x: pd.Timedelta(str(x))).astype('timedelta64[s]')
    # df_combined['MatRate'] = df_combined['Mats'] / df_combined['Duration']
    # df_combined['DiffClass'] = df_combined['Class'] + '-' + df_combined['Difficulty']
    return df

In [ ]:
def get_quality(df_num):
    df_min = df_num.transform(lambda x: x - x.min()).transform(lambda x: x/x.max())
    df_quality = df_min.to_frame('Quality')
    return df_quality

In [ ]:
def mad(df):
    const_med = df.median()
    df_med = abs(df - const_med)
    v_mad = df_med.median()
    return v_mad

In [ ]:
def get_ucb(row):
    div1 = 2 * math.log(row['total']) / row['count']
    ucb = math.sqrt(div1)
    return ucb + row['median']

In [ ]:
def get_ucb2(row):
    div1 = 3 * math.log(row['total']) / row['count']
    min1 = min(0.25, row['mad'] + div1)
    div2 = math.log(row['total']) / row['count']
    ucb = math.sqrt(div2 * min1)
    return ucb + row['median']

In [ ]:
def save_excel(df,col):
    dfgb = df.groupby([col])
    dfgb = dfgb['Quality'].agg(['count','median', mad]).reset_index()
    return dfgb

In [ ]:
df = get_frame('EQTime.xlsx')
df.tail(10)

In [ ]:
# reduce it
df = bd.reduce(df,'Project')
df['Duration'] = df['Duration'].apply(lambda x: pd.Timedelta(str(x))).astype('timedelta64[s]')
df_quality = get_quality(df['Duration'])
df = df.join(df_quality)

In [ ]:
df.tail(10)

In [ ]:
df.sort_values('Quality', ascending=False).head(10)

In [ ]:
dfgb = save_excel(df,'Project')

In [ ]:
df = df.merge(dfgb)
dfgb['total'] = dfgb['count'].sum()
dfgb['donext'] = dfgb.apply(get_ucb2, axis=1)
dfgb.sort_values('donext', ascending=False)